In [3]:
import requests
import json
import http.client
import numpy as np
import pymysql
import configparser


def moneyTrimmer(input_string):
    
    '''
    Hotel Price data would look like "$263 - $505".
    This method change the one to 263. (get minimum price)
    '''
    
    temp_list = input_string.split(" - ")
    return (temp_list[0])[1:].replace(",", "")


def hotelInformationGetter(input_data, offset):
    
    '''
    Get hotel data and extract information.
    '''
    
    # 파라미터 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")
    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
    api_url = ("/hotels/list?offset="
                 +str(offset)
                 +"&currency=USD&child_rm_ages=7%252C10&limit=30&checkin=2020-01-08&order=asc&lang=en_US&sort=recommended&nights=1&location_id="
                 + str(location_id)
                 +"&adults=1&rooms=1")
    #print(api_url)
    
    conn.request("GET", api_url, headers=headers)
    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    hotels_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            hotel_data = []
            hotel_data.append(one_data.get("name"))
            sub_category = one_data.get("subcategory_type")
            sub_categorys = ["hotel", "small_hotel", "hostel", "guest_house", "villa"]
            if not sub_category in sub_categorys:
                continue
            else:
                hotel_data.append(sub_category)                
            hotel_data.append(int(float(one_data.get("hotel_class"))))
            hotel_data.append(int(moneyTrimmer(one_data.get("price"))))
            hotel_data.append(cityId) # cityId
            hotels_data.append(hotel_data)
        except AttributeError as e:
            continue

    return hotels_data

def sqlController(hotel_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user, 
                           password=db_password,
                           db=db_name,
                           charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Hotel"
    sql_insert_hotel_information = "INSERT INTO City (continent, country, name) VALUES('Europe', 'France', 'Paris')"
    sql_insert_hotels_information = "INSERT INTO Hotel (name, category, subCategory, cost, CityId) VALUES(%s, %s, %s, %s, %s)"
    sql_delete = "DELETE FROM Hotel"
    sql_update_price = "UPDATE Hotel SET cost = %s WHERE name = %s"
    
    # 호텔 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    hotel_data = curs.fetchall() 
    #print("hotel_data", hotel_data) # 지금까지 DB에 있는 호텔 데이터
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 호텔 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_hotel_data = []
    overlapped_hotel_cost_data = []
    
    for info in hotel_info_list:
        if not dataOverlapChecker(hotel_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_hotel_data.append(info)
        else: # 겹치는 호텔의 경우 -> cost만 업데이트 한다
            overlapped_hotel_cost_data.append((info[3], info[0]))
    print("overlapped_hotel_cost_data", overlapped_hotel_cost_data)
    print("new_hotel_data", new_hotel_data)
    curs.executemany(sql_insert_hotels_information, new_hotel_data)
    curs.executemany(sql_update_price, overlapped_hotel_cost_data)
    
    #curs.execute(sql_delete) # 테이블 삭제
    #data = curs.fetchall() # 가져오기 (select에서 사용)
    #print(data)
    
    conn.commit() # RDS에 반영하기
    
def dataOverlapChecker(base_data, checking_info):
    '''
    겹치는 데이터가 있는지 여부를 확인하는 메서드
    '''
    
    if (checking_info[0],) in base_data:
        return True
    return False
            

'''
{"japan_tokyo_shinjuku" : 1066457, "cityId" : 2},
{"japan_tokyo_sibuya" : 1066456, "cityId" : 2},
{"taiwan_taipei" : 293913, "cityId" : 3},
{"vietnam_danang" : 298085, "cityId" : 4},
{"usa_newyork" : 60763, "cityId" : 1},
{"france_paris" : 187147, "cityId" : 0},
{"philippines_cebu" : 298460, "cityId" : 5}
[[1066457, 3], [1066456, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [298460, 6]]
'''

location_id_list = [[1066457, 3], [1066456, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [298460, 6]]
 
    
for offset in range(0, 90, 30):
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        hotel_info_list = hotelInformationGetter(location_id, offset)  
        sqlController(hotel_info_list)
    
    
# hotel_info_list = hotelInformationGetter(187147)
# sqlController(hotel_info_list)

[1066457, 3]
offset:  0
overlapped_hotel_cost_data [(83, 'Hotel Sunroute Higashi Shinjuku'), (141, 'Hotel Gracery Shinjuku'), (109, 'Shinjuku Washington Hotel Main'), (105, 'Tokyu Stay Shinjuku'), (107, 'Shinjuku Granbell Hotel'), (216, 'Keio Plaza Hotel Tokyo'), (96, 'Nishitetsu Inn Shinjuku'), (139, 'Hundred Stay Tokyo Shinjuku'), (74, 'Hotel Wing International Premium Tokyo Yotsuya'), (79, 'THE KNOT TOKYO Shinjuku'), (101, 'Nippon Seinenkan Hotel'), (83, 'APA Hotel Shinjuku Kabukicho Tower'), (144, 'Shinjuku Prince Hotel'), (69, 'Hotel Sunlite Shinjuku'), (82, 'APA Hotel Shinjuku Gyoemmae'), (86, 'Via inn Shinjuku'), (115, 'Citadines Central Shinjuku Tokyo'), (78, 'Hotel Wing International Shinjuku'), (97, 'Hotel Rose Garden Shinjuku'), (78, 'HOTEL MYSTAYS Nishi Shinjuku'), (113, 'Daiwa Roynet Hotel Nishi Shinjuku'), (267, 'Hilton Tokyo'), (291, 'Hyatt Regency Tokyo'), (84, 'Tokyo Plaza Hotel'), (129, 'Citadines Shinjuku Tokyo'), (139, 'RIHGA Royal Hotel Tokyo'), (72, 'APA Hotel Hig

KeyboardInterrupt: 

In [20]:
import requests
import json
import http.client
import numpy as np
import pymysql
import configparser

def moneyTrimmer(input_string):
    '''
    Food Price data would look like "$20 - $40".
    This method change the one to 30.
    This element will be inserted in cost column.
    '''
    temp_list = input_string.replace(",", "").split(" - ")
    if len(temp_list) == 1:
        return int(temp_list[0][1:])
    mean = (int)((int(temp_list[0][1:])+int(temp_list[1][1:]))/2)
    
    return mean

def priceLevelTrimmer(input_price_level):
    '''
    Price level data would look like $$ - $$$.
    This method change the one to 2.
    $ -> 1
    $$ - $$$ -> 2
    $$$$ -> 3
    This element will be inserted in grade column.
    '''
    if input_price_level == None:
        return None
    
    if input_price_level.count("$") == 1:
        return 1
    elif input_price_level.count("$") == 5:
        return 2
    else:
        return 3
    
def restaurantInformationGetter(input_data, offset):
    '''
    Get restaurant data and extract information.
    '''
    
    # 파라미터 변수 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")

    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
        
    conn.request("GET", "/restaurants/list?restaurant_tagcategory_standalone=10591&lunit=km&restaurant_tagcategory=10591&limit=30&prices_restaurants=10953%252C10955&restaurant_mealtype=10598%252C10599&currency=USD&lang=en_US&offset="
                 + str(offset)
                 + "&location_id="
                 + str(location_id),
                 headers=headers)

    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    restaurants_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            restaurant_data = []
            if one_data.get("name") == None:
                continue
            restaurant_data.append(one_data.get("name"))
            restaurant_data.append(priceLevelTrimmer(one_data.get("price_level"))) # grade
            if -1 != one_data.get("price", -1):
                restaurant_data.append(int(moneyTrimmer(one_data.get("price"))))  # cost
            else:
                restaurant_data.append(None) # cost
            restaurant_data.append(cityId) # cityId
            restaurants_data.append(restaurant_data)
        except AttributeError as e:
            continue
    return restaurants_data

def sqlController(food_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user ,password=db_password, 
                           db=db_name, charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Food"
    sql_insert_foods_information = "INSERT INTO Food (name, grade, cost, CityId) VALUES(%s, %s, %s, %s)"
    sql_delete = "DELETE FROM Food"
    sql_update_price = "UPDATE Food SET cost = %s WHERE name = %s"
    
    # Food 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    food_data = curs.fetchall() 
    #print("food_data", food_data)
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 Food 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_food_data = []
    overlapped_food_cost_data = []
    for info in food_info_list:
        if not dataOverlapChecker(food_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_food_data.append(info)
        else: # 겹치는 food의 경우 -> cost만 업데이트 한다
            overlapped_food_cost_data.append((info[3], info[0]))
    print("overlapped_food_cost_data", overlapped_food_cost_data)
    print("new_food_data", new_food_data)
    curs.executemany(sql_insert_foods_information, new_food_data)
    curs.executemany(sql_update_price, overlapped_food_cost_data)

    conn.commit() # RDS에 반영하기




location_id_list = [[1066457, 3], [1066456, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [298460, 6]]
 
for offset in range(0, 90, 30):
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        restaurant_info_list = restaurantInformationGetter(location_id, offset)  
        sqlController(restaurant_info_list)
    

[1066457, 3]
offset:  30
food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bo

food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bouchon Lyonnais',), ('Down

food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bouchon Lyonnais',), ('Down

food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bouchon Lyonnais',), ('Down

food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bouchon Lyonnais',), ('Down

...

food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bouchon Lyonnais',), ('Down

food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bouchon Lyonnais',), ('Down

food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bouchon Lyonnais',), ('Down

food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bouchon Lyonnais',), ('Down

food_data (('Hanamidori Shinjuku Mitsui Bldg.',), ('Nabezo Shinjuku 3 Chome',), ('Nabezo Shinjuku Meijidori',), ('Trattoria Dai Paesani',), ('Kyochabana, Shinjuku Toho Bldg',), ('Kuriya',), ('Rokkasen',), ('Katsukura Shinjuku Takashimaya',), ('Gyukatsu Motomura Shinjuku South exit Store',), ('Oedo Ayatori',), ('Shin Udon',), ('Shinjuku Gyoen Ramen Ouka',), ('Ikinari Steak Shinjuku Nichome',), ('Tsukiji Kiyomura Sushi-Zanmai Higashi Shinjuku-ten',), ('Jojoen Tokyo Opera City 53',), ('Nihonshu Genka Sakagura Shinjuku East Entrance',), ('Coco Ichibanya Shinjuku Station West Entrance',), ('Glutenfree Cafe Littlebird',), ('Nabezo Shibuya Center Street',), ('Himawari Zushi Shintoshin',), ('Nabezo Shibuya Koendori',), ('Sakuratei',), ('Han no Daidokoro Kadochika',), ('Gyozanofukuho',), ('AIN SOPH. Journey Shinjuku',), ('GYOPAO Gyoza Roppongi',), ('Gluten Free T’s Kitchen',), ('Shinjuku Kakekomi Gyoza kabukichou',), ('Tonkatsu Hamakatsu Nishi-Shinjuku',), ('Lugdunum Bouchon Lyonnais',), ('Down

In [70]:
# 연습

# {"japan_tokyo_shinjuku" : 1066457, "cityId" : 2},
# {"japan_tokyo_sibuya" : 1066456, "cityId" : 2},
# {"taiwan_taipei" : 293913, "cityId" : 3},
# {"vietnam_danang" : 298085, "cityId" : 4},
# {"usa_newyork" : 60763, "cityId" : 1},
# {"france_paris" : 187147, "cityId" : 0},
# {"philippines_cebu" : 298460, "cityId" : 5}

location_id_dict = [[1066457, 2], [1066456, 2], [293913, 3], [298085, 4], [60763, 1], [187147, 0], [298460, 5]]

    
for location_id in location_id_dict:
    print(location_id)
    # hotel_info_list = hotelInformationGetter(location_id)
    # sqlController(hotel_info_list)
    

[1066457, 2]
[1066456, 2]
[293913, 3]
[298085, 4]
[60763, 1]
[187147, 0]
[298460, 5]


In [4]:
# 연결 연습

import requests
import json
import http.client
import numpy as np
import pymysql

'''
Get restaurant data and extract information.
'''
# rapidapi 연결하기
conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
    'x-rapidapi-key': "164b57de3amsh8a6777213843ef5p15508djsn556f2053e4ab"
    }
# 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)

offset_number = 0

conn.request("GET", "/restaurants/list?restaurant_tagcategory_standalone=10591&lunit=km&restaurant_tagcategory=10591&limit=30&prices_restaurants=10953%252C10955&restaurant_mealtype=10598%252C10599&currency=USD&lang=en_US&offset="
             + str(30)
             + "&location_id="
             + str(298085),
             headers=headers)

res = conn.getresponse()

# response 읽기
data = res.read()

# response를 dictionary로 만들기
dict_data = json.loads(data)
print(dict_data) # 확인 출력문

# 딕셔너리 다듬기
dict_data_data = dict_data.get('data')


{'data': [{'location_id': '7057425', 'name': 'Limoncello', 'latitude': '16.066114', 'longitude': '108.22364', 'num_reviews': '894', 'timezone': 'Asia/Ho_Chi_Minh', 'location_string': 'Da Nang', 'photo': {'images': {'small': {'width': '150', 'url': 'https://media-cdn.tripadvisor.com/media/photo-l/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '150'}, 'thumbnail': {'width': '50', 'url': 'https://media-cdn.tripadvisor.com/media/photo-t/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '50'}, 'original': {'width': '1280', 'url': 'https://media-cdn.tripadvisor.com/media/photo-o/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '853'}, 'large': {'width': '1024', 'url': 'https://media-cdn.tripadvisor.com/media/photo-w/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '682'}, 'medium': {'width': '550', 'url': 'https://media-cdn.tripadvisor.com/media/photo-s/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '367'}}, 'is_blessed': True, 'uploaded_date': '2018-02-23T05:17:09-0500', 'caption': 